If you don't have Qiskit installed, you'll want to go ahead and run:

In [ ]:
pip install qiskit --upgrade

In [ ]:
pip install numpy --upgrade

And now to make sure everything is installed correctly:

In [ ]:
import qiskit.tools.jupyter
%qiskit_version_table
%qiskit_copyright

First, we'll load in the sequences we would like to compare.

In [ ]:
a = "AACCAGTCG"
b = "CACCGTAA"

Next, we need to generate a substitution matrix of matches and non-matches using the classical Smith-Waterman algorithm.

In [ ]:
import itertools
import numpy as np

# Initialize an (n+1)*(m+1) matrix
# Where n and m are len(a) and len(b) respectively
H = np.zeros((len(a) + 1, len(b) + 1), np.int)

# Compare every character in a against every character in b
# Matches (white cells) are denoted with 1
for i, j in itertools.product(range(1, H.shape[0]), range(1, H.shape[1])):
  if a[i - 1] == b[j - 1]:
    H[i, j] = 1

print("Before removing extra rows:")
print(H)

# TODO
# Verify this doesn't break if we swap a and b
for x in range(0, abs(H.shape[0] - H.shape[1])):
  H = np.delete(H, 0, 0)
  H = np.delete(H, 0, 1)

print("After removing extra rows:")
print(H)

print(H.shape[0])
print(H.shape[1])